# Setting

In [1]:
!pip install category_encoders
import category_encoders as ce

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 863.0 kB/s eta 0:00:00


In [2]:
import lightgbm as lgb

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/train.csv')
df_test_x = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/test.csv')
df_holidays = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/holidays_in_japan.csv')
df_venue = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/venue_information.csv')
df_match = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TOP_SIGNATE/Deroito_competition/match_reports.csv')

In [5]:
df_train_x = df_train.drop(['attendance'], axis=1)
df_train_y = df_train['attendance']

# Feature engieering

In [6]:
# 日付をdatetime型にする
df_holidays['holiday_date'] = pd.to_datetime(df_holidays['holiday_date'])
df_train_x['match_date'] = pd.to_datetime(df_train_x['match_date'])
df_test_x['match_date'] = pd.to_datetime(df_test_x['match_date'])

In [7]:
# 休日なら1、平日なら0のholidays列をデータフレームに追加
df_train_x['holidays'] =df_train_x['match_date'].dt.dayofweek.apply(lambda x: 1 if(x in df_holidays['holiday_date'].values or x in [5,6]) else 0)
df_test_x['holidays'] =df_test_x['match_date'].dt.dayofweek.apply(lambda x: 1 if(x in df_holidays['holiday_date'].values or x in [5,6]) else 0)

In [8]:
# sectionとroundからsection_roundを作る
df_train_x['section_round'] = df_train_x['section'] + '_' + df_train_x['round']
df_test_x['section_round'] = df_test_x['section'] + '_' + df_test_x['round']
df_train_x = df_train_x.drop(['section', 'round'], axis=1)
df_test_x = df_test_x.drop(['section', 'round'], axis=1)

In [9]:
# kick_off_timeをhourだけにする
df_train_x['kick_off_time'] = pd.to_datetime(df_train_x['kick_off_time'], format='%H:%M').dt.hour
df_test_x['kick_off_time'] = pd.to_datetime(df_test_x['kick_off_time'], format='%H:%M').dt.hour
df_train_x.head()

,id,match_date,kick_off_time,home_team,away_team,venue,weather,temperature,humidity,broadcasters,holidays,section_round
0,9190,2006-03-04,16,G大阪,浦和,万博記念競技場,晴,8.3,40,NHK総合/J SPORTS(録),1,第1節_第1日
1,9191,2006-03-05,13,甲府,清水,山梨県小瀬スポーツ公園陸上競技場,晴,12.9,28,山梨放送/テレビ静岡(録)/J SPORTS(録),1,第1節_第2日
2,9192,2006-03-05,13,FC東京,大分,味の素スタジアム,晴,12.1,35,BS-i/MXテレビ(録)/J SPORTS(録),1,第1節_第2日
3,9193,2006-03-05,14,磐田,福岡,静岡スタジアムエコパ,晴,11.6,42,J SPORTS,1,第1節_第2日
4,9194,2006-03-05,14,名古屋,C大阪,名古屋市瑞穂陸上競技場,晴,13.1,32,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),1,第1節_第2日


In [10]:
# 都道府県を見つける関数
def extract_prefecture(address):
  if '県' in address:
    return address[:address.find('県')+1]
  elif '道' in address:
    return address[:address.find('道')+1]
  elif '府' in address:
    return address[:address.find('府')+1]
  elif '都' in address:
    return address[:address.find('都')+1]
  else:
    return None

In [11]:
# df_venueに都道府県を示すprefecture列を追加
df_venue['prefecture'] = df_venue['address'].apply(extract_prefecture)
df_venue.head()

,venue,capacity,address,prefecture
0,札幌ドーム,38794,北海道札幌市豊平区羊ヶ丘1,北海道
1,埼玉スタジアム2002,62010,埼玉県さいたま市緑区中野田500,埼玉県
2,味の素スタジアム,47851,東京都調布市西町376−3,東京都
3,町田GIONスタジアム,15320,東京都町田市野津田町2035,東京都
4,日産スタジアム,71624,神奈川県横浜市港北区小机町3300,神奈川県


In [13]:
# データにvenueをマージして、capacity,prefecture列を追加
df_train_x = df_train_x.merge(df_venue[['venue', 'prefecture', 'capacity']], on='venue', how='left')
df_test_x = df_test_x.merge(df_venue[['venue', 'prefecture', 'capacity']], on='venue', how='left')

In [16]:
# df_matchとデータをidでマージする。選手情報の追加
df_train_x = pd.merge(df_train_x, df_match, on='id', how='left')
df_test_x = pd.merge(df_test_x, df_match, on='id', how='left')
df_train_x.head()

,id,match_date,kick_off_time,home_team,away_team,venue,weather,temperature,humidity,broadcasters,...,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11
0,9190,2006-03-04,16,G大阪,浦和,万博記念競技場,晴,8.3,40,NHK総合/J SPORTS(録),...,2 坪井慶介 DF,4 田中マルクス闘莉王 DF,20 堀之内聖 DF,6 山田暢久 MF,17 長谷部誠 MF,13 鈴木啓太 MF,8 三都主アレサンドロ MF,18 小野伸二 MF,10 ポンテ FW,21 ワシントン FW
1,9191,2006-03-05,13,甲府,清水,山梨県小瀬スポーツ公園陸上競技場,晴,12.9,28,山梨放送/テレビ静岡(録)/J SPORTS(録),...,25 市川大祐 DF,26 青山直晃 DF,4 高木和道 DF,3 山西尊裕 DF,13 兵働昭弘 MF,7 伊東輝悦 MF,16 枝村匠馬 MF,10 藤本淳吾 MF,18 チョジェジン FW,17 マルキーニョス FW
2,9192,2006-03-05,13,FC東京,大分,味の素スタジアム,晴,12.1,35,BS-i/MXテレビ(録)/J SPORTS(録),...,2 三木隆司 DF,4 深谷友基 DF,22 上本大海 DF,5 エジミウソン MF,11 トゥーリオ MF,6 梅田高志 MF,17 根本裕一 MF,26 内村圭宏 MF,13 高松大樹 FW,9 オズマール FW
3,9193,2006-03-05,14,磐田,福岡,静岡スタジアムエコパ,晴,11.6,42,J SPORTS,...,22 中村北斗 DF,4 金古聖司 DF,5 千代反田充 DF,3 アレックス DF,7 宮崎光平 MF,6 布部陽功 MF,8 ホベルト MF,14 古賀誠史 MF,27 田中佑昌 FW,36 グラウシオ FW
4,9194,2006-03-05,14,名古屋,C大阪,名古屋市瑞穂陸上競技場,晴,13.1,32,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),...,2 ブルーノクアドロス DF,5 前田和哉 DF,3 柳本啓成 DF,4 河村崇大 MF,23 下村東美 MF,10 ピンゴ MF,6 ゼカルロス MF,8 森島寛晃 FW,20 西澤明訓 FW,9 古橋達弥 FW


In [17]:
# obj型とそれ以外を分ける
df_train_x_obj = df_train_x.select_dtypes(include='object')
df_test_x_obj = df_test_x.select_dtypes(include='object')
df_train_x_num = df_train_x.select_dtypes(exclude='object')
df_test_x_num = df_test_x.select_dtypes(exclude='object')

In [18]:
# objをエンコーダーで数字を割り振る
list_cols = df_train_x_obj.columns
encoder = ce.OrdinalEncoder(cols=list_cols)
df_train_x_obj_enc = encoder.fit_transform(df_train_x_obj)
df_test_x_obj_enc = encoder.transform(df_test_x_obj)

In [19]:
df_train_x_coc = pd.concat([df_train_x_num, df_train_x_obj_enc], axis=1)
df_test_x_coc = pd.concat([df_test_x_num, df_test_x_obj_enc], axis=1)

In [20]:
# 何月かを示すmatch_monthを作成
df_train_x_coc['match_month'] = df_train_x_coc['match_date'].dt.month
df_test_x_coc['match_month'] = df_test_x_coc['match_date'].dt.month
df_train_x_coc.head()

,id,match_date,kick_off_time,temperature,humidity,holidays,capacity_x,capacity_y,home_team_score,away_team_score,...,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11,match_month
0,9190,2006-03-04,16,8.3,40,1,21000,21000,1,1,...,1,1,1,1,1,1,1,1,1,3
1,9191,2006-03-05,13,12.9,28,1,15859,15859,0,2,...,2,2,2,2,2,2,2,2,2,3
2,9192,2006-03-05,13,12.1,35,1,47851,47851,2,0,...,3,3,3,3,3,3,3,3,3,3
3,9193,2006-03-05,14,11.6,42,1,51697,51697,1,1,...,4,4,4,4,4,4,4,4,4,3
4,9194,2006-03-05,14,13.1,32,1,20223,20223,3,2,...,5,5,5,5,5,5,5,5,5,3


In [21]:
df_train_x_coc = df_train_x_coc.drop(['id', 'match_date'], axis=1)
df_test_x_coc = df_test_x_coc.drop(['id', 'match_date'], axis=1)

# prediction

In [22]:
# LightGBMのモデル構築
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [23]:
# 評価用と訓練用にデータを分ける
x_train_pra, x_train_val, y_train_pra, y_train_val = train_test_split(df_train_x_coc, df_train_y, test_size=0.2)

In [24]:
# 評価用と訓練用のデータをlgbに登録
train_data = lgb.Dataset(x_train_pra, label=y_train_pra)
eval_data = lgb.Dataset(x_train_val, label=y_train_val)

In [25]:
# 学習
model = lgb.train(params, train_data, valid_sets=[eval_data], num_boost_round=1000)

In [26]:
# 評価を見てみる
y_pred = model.predict(x_train_val)
score = r2_score(y_train_val, y_pred)
print(score)

0.7948418553367366


In [27]:
y_test_pred = model.predict(df_test_x_coc)

In [28]:
submission = pd.DataFrame({'id': df_test_x['id'], 'attendance': y_test_pred})
submission.to_csv('Deroito_J_league_07.csv', index=False, header=False)